<a href="https://colab.research.google.com/github/anhle/tensorFlow2.x/blob/master/Tensorflow_2_overview.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#TensorFlow Overview 

A quick look at some MNIST examples to get familiar with the core features of TensorFlow 2.0:

- tf.keras: A high-level, object-oriented API for fast prototyping of deep learning models
- tf.GradientTape: Records gradients on-the-fly for automatic differentiation and backprop
- tf.train: Optimizers for training and checkpoints for exporting models


#Train MNIST with a convolutional network:

To get started, import the TensorFlow library into your program:

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

!pip install -q tensorflow==2.0.0-alpha0
import tensorflow as tf
import tensorflow_datasets as tfds
from   tensorflow.python.ops import summary_ops_v2
from   tensorflow import keras
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Flatten, Conv2D

import  os
import  time
import  numpy as np

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

    100% |████████████████████████████████| 79.9MB 403kB/s 
    100% |████████████████████████████████| 3.0MB 12.4MB/s 
    100% |████████████████████████████████| 61kB 29.3MB/s 
    100% |████████████████████████████████| 419kB 23.9MB/s 


Load and prepare the MNIST dataset. Convert the samples from integers to floating-point numbers:

In [0]:
def convert_types(image, label):
  image = tf.cast(image, tf.float32)
  image /= 255
  return image, label

dataset, info = tfds.load('mnist', data_dir='gs://tfds-data/datasets', with_info=True, as_supervised=True)
mnist_train, mnist_test = dataset['train'], dataset['test']

mnist_train = mnist_train.map(convert_types).shuffle(10000).batch(32)
mnist_test = mnist_test.map(convert_types).batch(32)

Build the tf.keras model using the Keras model subclassing API.  Choose an optimizer and loss function used for training:

In [0]:
class MyModel(Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.conv1 = Conv2D(32, 3, activation='relu')
    self.flatten = Flatten()
    self.d1 = Dense(128, activation='relu')
    self.d2 = Dense(10, activation='softmax')

  def call(self, x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)

model = MyModel()

loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

optimizer = tf.keras.optimizers.Adam()


Select metrics to measure the loss and the accuracy of the model. These metrics accumulate the values over epochs and then print the overall result.

In [0]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

Train and evaluate model:
* Train the model using tf.GradientTape:

In [0]:
@tf.function
def train_step(image, label):
  with tf.GradientTape() as tape:
    predictions = model(image)
    loss = loss_object(label, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(label, predictions)
  
@tf.function
def test_step(image, label):
  predictions = model(image)
  t_loss = loss_object(label, predictions)

  test_loss(t_loss)
  test_accuracy(label, predictions)

In [7]:
# Where to save checkpoints, tensorboard summaries, etc.
MODEL_DIR = '/mnist'

checkpoint_dir = os.path.join(MODEL_DIR, 'checkpoints')
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt')

checkpoint = tf.train.Checkpoint(model=model, optimizer=optimizer)

# Restore variables on creation if a checkpoint exists.
#checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))
    
EPOCHS = 5

for epoch in range(EPOCHS):
  for image, label in mnist_train:
    start = time.time()
    train_step(image, label)
    end = time.time()
    

  for test_image, test_label in mnist_test:
    test_step(test_image, test_label)

  template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  print (template.format(epoch+1,
                         train_loss.result(),
                         train_accuracy.result()*100,
                         test_loss.result(),
                         test_accuracy.result()*100))
  
  checkpoint.save(checkpoint_prefix)
  print('saved checkpoint.')

export_path = os.path.join(MODEL_DIR, 'export')
tf.saved_model.save(model, export_path)
print('saved SavedModel for exporting.')

Epoch 1, Loss: 0.13342660665512085, Accuracy: 95.91500091552734, Test Loss: 0.06172272190451622, Test Accuracy: 97.93999481201172
saved checkpoint.
Epoch 2, Loss: 0.08763611316680908, Accuracy: 97.2925033569336, Test Loss: 0.0593315027654171, Test Accuracy: 98.06999969482422
saved checkpoint.
Epoch 3, Loss: 0.06628486514091492, Accuracy: 97.94166564941406, Test Loss: 0.05797857046127319, Test Accuracy: 98.15999603271484
saved checkpoint.
Epoch 4, Loss: 0.05331886559724808, Accuracy: 98.3387451171875, Test Loss: 0.05735132098197937, Test Accuracy: 98.21749877929688
saved checkpoint.
Epoch 5, Loss: 0.0444309264421463, Accuracy: 98.61067199707031, Test Loss: 0.059119150042533875, Test Accuracy: 98.26399993896484
saved checkpoint.
saved SavedModel for exporting.
